In [4]:
import pandas as pd
from pandarallel import pandarallel
from typing import defaultdict

In [9]:
data_path = '..\\data\\'

In [3]:
clients_cleaned = pd.read_feather(data_path + '\\interim\\clnts.frt')

In [16]:
def columns_report(data: pd.DataFrame) -> None:

    print('Rows:', data.shape[0], end='\n'*2)

    stats = defaultdict(list)
    for col in data.columns:
        stats['Column'].append(col)
        stats['Unique'].append(data[col].nunique())
        stats['Duplicates'].append(data[col].duplicated().sum())
        stats['Missing'].append(data[col].isna().sum())
        stats['Missing%'].append(data[col].isna().mean() * 100)
        stats['HitRate%'].append((1 - data[col].isna().mean()) * 100)
    stats = pd.DataFrame(stats)

    print(stats.to_string(index=False))

### Соотношение среднедушевых денежных доходов населения с величиной прожиточного минимума (процент)

In [5]:
avgpercapinc_subsistmin_ratio = pd.read_excel(data_path + 'external\\соотношение_среднедушевых_доходов_с_прожиточным_минимумом_по_субъектам.xls',
                                              sheet_name='long_fmt')
print(avgpercapinc_subsistmin_ratio.shape)

(5357, 4)


In [6]:
avgpercapinc_subsistmin_ratio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5357 entries, 0 to 5356
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Субъект   5357 non-null   object 
 1   Год       5357 non-null   int64  
 2   Квартал   5357 non-null   object 
 3   Значение  5357 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 167.5+ KB


In [7]:
columns_report(avgpercapinc_subsistmin_ratio)

Rows: 5357

  Column  Unique  Duplicates  Missing  Missing%  HitRate%
 Субъект     102        5255        0       0.0     100.0
     Год      26        5331        0       0.0     100.0
 Квартал       4        5353        0       0.0     100.0
Значение    2393        2964        0       0.0     100.0


In [8]:
subjects = ['край', 'область', 'республика', 'москва', 'санкт-петербург', 'российская федерация']
stop_words = subjects[:3]


def rm_stop_words(s: str, stop_words=stop_words):

    return ' '.join(w for w in s.split() if w not in stop_words)

In [9]:
def prep_fedstat_geo(data: pd.DataFrame, subs_to_maintain: list,
                     geo_col: str = 'Субъект') -> pd.DataFrame:

    pandarallel.initialize(progress_bar=False, nb_workers=12)

    data[geo_col] = data[geo_col]\
        .str.strip()\
        .str.lower()\
        .str.replace(r'\s*\(.*\)', '', regex=True)\
        .str.replace(r' - .*', '', regex=True)
    data = data.loc[
        data[geo_col].str.contains('|'.join(subs_to_maintain))].reset_index(drop=True)
    data[geo_col] = data[geo_col].parallel_apply(
        rm_stop_words)\
        .str.strip()

    return data

In [10]:
avgpercapinc_subsistmin_ratio = prep_fedstat_geo(avgpercapinc_subsistmin_ratio, subjects)

clients_cleaned['geo'] = clients_cleaned.geo\
    .str.strip()\
    .str.lower()\
    .str.replace(r'\s*\(.*\)', '', regex=True)\
    .parallel_apply(rm_stop_words)\
    .str.strip()

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [11]:
avgpercapinc_subsistmin_ratio['Дата'] = avgpercapinc_subsistmin_ratio.Год.astype(str) + avgpercapinc_subsistmin_ratio.Квартал

avgpercapinc_subsistmin_ratio.drop(columns=['Год', 'Квартал'], inplace=True)

In [12]:
avgpercapinc_subsistmin_ratio.rename(columns={
    'Субъект': 'geo',
    'Значение': 'avgpercapinc_subsistmin_ratio',
    'Дата': 'date'
}, inplace=True)

In [13]:
set(avgpercapinc_subsistmin_ratio.geo.unique()).intersection(
    set(clients_cleaned.geo.unique())
)

{'адыгея',
 'алтай',
 'алтайский',
 'амурская',
 'архангельская',
 'астраханская',
 'башкортостан',
 'белгородская',
 'брянская',
 'бурятия',
 'владимирская',
 'волгоградская',
 'вологодская',
 'воронежская',
 'дагестан',
 'ивановская',
 'ингушетия',
 'иркутская',
 'кабардино-балкарская',
 'калининградская',
 'калмыкия',
 'калужская',
 'карачаево-черкесская',
 'карелия',
 'кемеровская',
 'кировская',
 'коми',
 'костромская',
 'краснодарский',
 'красноярский',
 'курганская',
 'курская',
 'ленинградская',
 'липецкая',
 'магаданская',
 'марий эл',
 'мордовия',
 'москва',
 'московская',
 'мурманская',
 'нижегородская',
 'новгородская',
 'новосибирская',
 'омская',
 'оренбургская',
 'орловская',
 'пензенская',
 'пермский',
 'приморский',
 'псковская',
 'ростовская',
 'рязанская',
 'самарская',
 'санкт-петербург',
 'саратовская',
 'саха',
 'сахалинская',
 'свердловская',
 'северная осетия-алания',
 'смоленская',
 'ставропольский',
 'тамбовская',
 'татарстан',
 'тверская',
 'томская',
 'тульс

In [15]:
# avgpercapinc_subsistmin_ratio.to_feather(data_path + 'interim\\avgpercapinc_subsistmin_ratio.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


### Ожидаемая продолжительность жизни при рождении

In [33]:
lifeexp_at_birth = pd.read_excel(data_path + 'external\\ожидаемая_продолжительность_жизни_при_рождении.xls',
                                 sheet_name='long_fmt')
print(lifeexp_at_birth.shape)

(3106, 3)


In [34]:
lifeexp_at_birth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3106 entries, 0 to 3105
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Субъект   3106 non-null   object 
 1   Год       3106 non-null   int64  
 2   Значение  3106 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 72.9+ KB


In [35]:
columns_report(lifeexp_at_birth)

Rows: 3106

  Column  Unique  Duplicates  Missing  Missing%  HitRate%
 Субъект     106        3000        0       0.0     100.0
     Год      33        3073        0       0.0     100.0
Значение     777        2329        0       0.0     100.0


In [36]:
lifeexp_at_birth = prep_fedstat_geo(lifeexp_at_birth, subjects)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [37]:
lifeexp_at_birth.rename(columns={
    'Субъект': 'geo',
    'Значение': 'lifeexp_at_birth',
    'Год': 'year'
}, inplace=True)

In [38]:
# lifeexp_at_birth.to_feather(data_path + 'interim\\lifeexp_at_birth.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


### Возрастной коэффициент смертности (промилле (0,1 процента))

In [39]:
agespec_death_rate = pd.read_excel(data_path + 'external\\возрастные_коэффициенты_смертности.xls',
                                   sheet_name='long_fmt')
print(agespec_death_rate.shape)

(1169, 3)


In [40]:
agespec_death_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1169 entries, 0 to 1168
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Год                                 1169 non-null   int64  
 1   Субъект                             1169 non-null   object 
 2   Возрастные коэффициенты смертности  1169 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 27.5+ KB


In [41]:
columns_report(agespec_death_rate)

Rows: 1169

                            Column  Unique  Duplicates  Missing  Missing%  HitRate%
                               Год      33        1136        0       0.0     100.0
                           Субъект      98        1071        0       0.0     100.0
Возрастные коэффициенты смертности     194         975        0       0.0     100.0


In [42]:
agespec_death_rate = prep_fedstat_geo(agespec_death_rate, subjects)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [43]:
agespec_death_rate.rename(columns={
    'Субъект': 'geo',
    'Возрастные коэффициенты смертности': 'agespec_death_rate',
    'Год': 'year'
}, inplace=True)

In [44]:
# agespec_death_rate.to_feather(data_path + 'interim\\agespec_death_rate.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


### Число умерших за год (человек, значение показателя за год)

In [46]:
deaths_per_year = pd.read_excel(data_path + 'external\\число_умерших_за_год.xls',
                                sheet_name='long_fmt')
print(deaths_per_year.shape)

(3321, 3)


In [47]:
deaths_per_year.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Субъект   3321 non-null   object
 1   Год       3321 non-null   int64 
 2   Значение  3321 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 78.0+ KB


In [48]:
columns_report(deaths_per_year)

Rows: 3321

  Column  Unique  Duplicates  Missing  Missing%  HitRate%
 Субъект     117        3204        0       0.0     100.0
     Год      33        3288        0       0.0     100.0
Значение    3186         135        0       0.0     100.0


In [49]:
deaths_per_year = prep_fedstat_geo(deaths_per_year, subjects)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [50]:
deaths_per_year.rename(columns={
    'Субъект': 'geo',
    'Значение': 'deaths_per_year',
    'Год': 'year'
}, inplace=True)

In [52]:
# deaths_per_year.to_feather(data_path + 'interim\\deaths_per_year.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


### Индекс потребительских цен на товары и услуги к концу предыдущего месяца

In [116]:
cpi_data = pd.read_excel(data_path + 'external\\ипц.xlsx',
                         sheet_name='long_fmt')
print(cpi_data.shape)

(377, 3)


In [117]:
cpi_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377 entries, 0 to 376
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Месяц   377 non-null    int64  
 1   Год     377 non-null    int64  
 2   ИПЦ     377 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 9.0 KB


In [118]:
columns_report(cpi_data)

Rows: 377

Column  Unique  Duplicates  Missing  Missing%  HitRate%
 Месяц      12         365        0       0.0     100.0
   Год      32         345        0       0.0     100.0
   ИПЦ     228         149        0       0.0     100.0


In [119]:
cpi_data.rename(columns={
    'Месяц': 'month',
    'ИПЦ': 'cpi',
    'Год': 'year'
}, inplace=True)

In [120]:
cpi_data['date'] = (pd.to_datetime(cpi_data.year.astype(str) + '.' + cpi_data.month.astype(str) + '.' + '01') +\
    pd.tseries.offsets.MonthEnd(0)).dt.to_period('Q')
cpi_data = cpi_data.groupby('date', as_index=False).cpi.mean()
cpi_data.reset_index(drop=True, inplace=True)

In [122]:
# cpi_data.to_feather(data_path + 'interim\\cpi.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


### Инфляция по месяцам в годовом исчислении

In [12]:
# парсинг данных об инфляции
# inflation_data = pd.read_html(
#     'https://xn----ctbjnaatncev9av3a8f8b.xn--p1ai/%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86%D1%8B-%D0%B8%D0%BD%D1%84%D0%BB%D1%8F%D1%86%D0%B8%D0%B8')[0]
# inflation_data.to_excel(data_path + 'external\\инфляция.xlsx', index=False)

In [13]:
inflation_data = pd.read_excel(data_path + 'external\\инфляция.xlsx',
                               sheet_name='long_fmt')
print(inflation_data.shape)

(381, 3)


In [14]:
inflation_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381 entries, 0 to 380
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Год       381 non-null    int64  
 1   Месяц     381 non-null    int64  
 2   Инфляция  381 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 9.1 KB


In [17]:
columns_report(inflation_data)

Rows: 381

  Column  Unique  Duplicates  Missing  Missing%  HitRate%
     Год      33         348        0       0.0     100.0
   Месяц      12         369        0       0.0     100.0
Инфляция     350          31        0       0.0     100.0


In [18]:
inflation_data.rename(columns={
    'Месяц': 'month',
    'Инфляция': 'inflation',
    'Год': 'year'
}, inplace=True)

In [19]:
inflation_data['date'] = (pd.to_datetime(inflation_data.year.astype(str) + '.' + inflation_data.month.astype(str) + '.' + '01') +\
    pd.tseries.offsets.MonthEnd(0)).dt.to_period('Q')
inflation_data = inflation_data.groupby('date', as_index=False).inflation.mean()
inflation_data.reset_index(drop=True, inplace=True)

In [21]:
# inflation_data.to_feather(data_path + 'interim\\inflation.frt')

d:\Programs\Python\Python311\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
